In [12]:
import pandas as pd
import numpy as np
import copy
import sys
import os
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
curr_dir = os.getcwd()
data_folder = os.path.join(curr_dir,"dados_camara_todos.csv")
dataset = pd.read_csv(data_folder)
#dataset = pd.read_csv("\\Users\\ASUS\\Documents\\IST\\5ºAno\\CT15Mn-150818_101018\\dados_camara.csv")#dados mais recentes
#tomtom = pd.read_csv("\\Users\\ASUS\\Documents\\IST\\5ºAno\\tomtom_data.csv")
#dataset = pd.read_csv("\\Users\\ASUS\\Documents\\IST\\5ºAno\\periodic_data.csv") #dados periodicos gerados automaticamente
#dataset = pd.read_csv("\\Users\\ASUS\\Documents\\IST\\5ºAno\\dados_old.csv") #dados mais antigos
dataset['unique_id'] = dataset.Zona.astype(str) + '_' + dataset.ID_Espira.astype(str)
dataset['unique_id'] = dataset['unique_id'].str.lower()
dataset_uid = dataset[(dataset["unique_id"] == "4_ct4")]

dataset_uid = dataset_uid.drop(columns=["Zona","Contadores","ID_Espira","unique_id"])
dt2 = copy.deepcopy(dataset_uid)
#dataset.sort_values(['Data'],ascending=True).groupby('Data').reset_index()

dataset_uid = dataset_uid.groupby('Data').apply(lambda x: x.reset_index())

msk = np.random.rand(len(dataset_uid)) < 0.7
train_df = dataset_uid[msk]
test_df = dataset_uid[~msk]

dt2.to_csv("lstm/limited_data.csv", sep= ',', index=False)
#train_set
train_df = train_df.drop(columns=["index"])

train_df.to_csv("lstm/train.csv", sep= ',', index=False)

#test_set
test_df = test_df.drop(columns=["index"])
test_df.to_csv("lstm/test.csv", sep= ',', index=False)

#full set
dataset_uid = dataset_uid.drop(columns=["index"])
dataset_uid.to_csv("lstm/dados_nn.csv", sep=',', index=False)

In [15]:
from scipy import stats
import statistics


import numpy as np

train_cp = copy.deepcopy(train_df)
test_cp = copy.deepcopy(test_df)

def smooth_data(train_cp,filename):
    data = train_cp.iloc[:,0]
    print(len(data.values))
    train_cp = train_cp.drop(columns=["Data"])
    train_cp = train_cp.values

    #train_cp = train_cp.astype('float32')

    avg_list = []
    std_list = []
    for i in range(len(train_cp[0])):
        curr_avg = sum(train_cp[:,i])/len(train_cp[:,i])
        avg_list.append(curr_avg)
        std_list.append(statistics.stdev(train_cp[:,i]))
    #FIX sera que e preciso guardar os valores antigos e usá-los para fazer os updates? 
    #agora esta a fazer uma especie de moving average, utilizando os valores novos nos updates seguintes


    number_of_changes= 0
    for row in range(len(train_cp)):
        
        for column in range(len(train_cp[0])):
            if (train_cp[row][column] > (avg_list[column] + 2*std_list[column])):
                old_value = train_cp[row][column]
                if column == 0:
                    previous_t = train_cp[row,-1]
                    next_t = train_cp[row,column+1]
                    train_cp[row,column] = (previous_t + next_t)/2
                    number_of_changes+=1
                if column == 95:
                    previous_t = train_cp[row,column-1]
                    next_t = train_cp[row,0]
                    train_cp[row,column] = (previous_t + next_t)/2
                    number_of_changes+=1
                else:
                    previous_t = train_cp[row,column-1]
                    next_t = train_cp[row,column+1]
                    train_cp[row,column] = (previous_t + next_t)/2
                    number_of_changes+=1
                new_value = train_cp[row][column]
    print("Number of changes: " + str(number_of_changes))
                
    new_d = data.values.reshape(len(data.values),1)
    test = np.concatenate((new_d,train_cp),axis=1)
    pd.DataFrame(test).to_csv("lstm/" + str(filename) + ".csv", sep=',', index=False)
    return test, train_cp
smooth_train, df_train= smooth_data(train_cp, "train_2")
smooth_test, df_test = smooth_data(test_cp, "test_2")

1464
Number of changes: 3205
640
Number of changes: 1370


In [18]:
def transform_data(in_file, out_file, nrows=-1):
    in_file = open(str(in_file),"r")
    next(in_file)
    out_file = open(str(out_file),"w")
    out_file.write("t-3,t-2,t-1,Y\n") #header
    k = 0    
    lines = in_file.readlines()
    for line in lines:
        line = line.split(",")
        line = line[1:]
        line[-1] = line[-1].replace("\n","") #last data record has a \n
        its = [iter(line), iter(line[1:]), iter(line[2:]),iter(line[3:])] #Construct the pattern for longer windowss
        x = list(zip(*its))
        if (k == nrows):
            break
            
        k+=1 
#print(x)
    #j = 0
    #while(j<50): #this cycle was for creating a mock dataset with repeated data for testing purposes
        for i in x:
        #print(i[0],i[1],i[2],i[3])
            out_file.write(i[0] + "," + i[1] +"," + i[2] +","+ i[3])
            out_file.write("\n")
     #   j+=1
    
    in_file.close()
    out_file.close()
transform_data("lstm/train_2.csv","lstm/train_formatted.csv")


In [28]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras import optimizers
import sys


dataframe = pd.read_csv('lstm/train_formatted.csv')
dataset = dataframe.values
train_set = dataset.astype('float32')


def create_dataset(data):
    X,Y = [],[]
    for i in data:
        X.append(i[:3])
        Y.append(i[3])
    return numpy.array(X), numpy.array(Y)
print(len(train_set))
trainX, trainY = create_dataset(train_set)
print(trainX)
print(trainY)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
# choose a number of time steps
n_steps = 3
trainX = trainX.reshape((trainX.shape[0], trainX.shape[1], n_features))
adam=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer=adam, loss='mse')
# fit model
model.fit(trainX, trainY, epochs=200, verbose=1)


136152
[[19. 17. 15.]
 [17. 15. 10.]
 [15. 10.  9.]
 ...
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
[10.  9. 18. ...  0.  0.  0.]
Epoch 1/200
136152/136152 [==============================] - 81s 596us/step - loss: 67.3682
Epoch 2/200
136152/136152 [==============================] - 75s 554us/step - loss: 55.4721
Epoch 3/200
136152/136152 [==============================] - 80s 587us/step - loss: 55.0271
Epoch 4/200
 26784/136152 [====>.........................] - ETA: 1:11 - loss: 54.2250

KeyboardInterrupt: 